In [16]:
import numpy as np
import pandas as pd
import torch
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
def load_result(csv_file, res_file, batch_size):
    df = pd.read_csv(csv_file)
    res_file = torch.load(res_file)
    num_batch = len(res_file["Output"])
    
    token_len_list = np.array([res_file["Output"][i].shape[1] for i in range(num_batch)])
    batch_list     = [ batch_size for i in range(num_batch)]
    df_selected    =df# df.loc[:, ['duration', 'emissions','emissions_rate', 'cpu_power', 'gpu_power', 'ram_power',
                                # 'cpu_energy','gpu_energy', 'ram_energy', 'energy_consumed', 'cpu_count',
                                # 'cpu_model','gpu_count', 'gpu_model',  'ram_total_size']]

    energy_per_prompt = (df_selected["gpu_energy"]/batch_size).to_numpy()
    # print(token_len_list)
    # print(energy_per_prompt)
    energy_per_prompt_j = (df_selected["gpu_energy"]*3.6e6/batch_size).to_numpy()
    energy_per_token  = energy_per_prompt/token_len_list
    energy_per_token_j  = energy_per_prompt_j/token_len_list

    throughput  = (token_len_list*batch_size/df_selected["duration"]).to_numpy()
    
    df_selected.insert(3, "Energy/Prompt (kWh/Prop)", energy_per_prompt, True)
    df_selected.insert(4, "Energy/Token (kWh/Token)", energy_per_token, True)
    df_selected.insert(5, "Throughput (Token/s)", throughput, True)
    df_selected.insert(6, "Tokens", token_len_list, True)
    df_selected.insert(7, "Batch Size", batch_list, True)
    df_selected.insert(8, "Energy/Prompt (J/Prop)", energy_per_prompt_j, True)
    df_selected.insert(9, "Energy/Token (J/Token)", energy_per_token_j, True)
    # print(token_len_list,token_len_list*batch_size)
    return df_selected

In [57]:
csv_file_l4 = "emission/emissions_L4exp5_Llama-2-7b-chat-hf_batch_1.csv"
res_file_l4 = "emission/inference_result_L4exp5_Llama-2-7b-chat-hf_batch_1.pth"

csv_file_a100 = "emission/emissions_A100exp5_Llama-2-7b-chat-hf_batch_1.csv"
res_file_a100 = "emission/inference_result_A100exp5_Llama-2-7b-chat-hf_batch_1.pth"
batch_size = 1

df_llama7b_l4 = load_result(csv_file_l4, res_file_l4, batch_size)
df_llama7b_l4['GPU'] = 'L4'
df_llama7b_l4['Energy(J)'] = df_llama7b_l4['energy_consumed']*3.6e6*1.2

df_llama7b_a100 = load_result(csv_file_a100, res_file_a100, batch_size)
df_llama7b_a100['GPU'] = 'A100'
df_llama7b_a100['Energy(J)'] = df_llama7b_a100['energy_consumed']*3.6e6*1.2


In [62]:
df_llama7b_measurement = pd.concat([df_llama7b_l4,df_llama7b_a100], ignore_index=True)
df_llama7b_measurement['Source'] = 'Llama'

In [ ]:
# Add a column to indicate the origin of each row
df_llama7b_l4['source'] = 'L4'
df_llama7b_a100['source'] = 'A100'

# Concatenate again with the source column
df_llama7b_measurement = pd.concat([df_llama7b_l4, df_llama7b_a100], ignore_index=True)

# Create the box plot
# plt.figure(figsize=(12, 8))
sns.boxplot(x='source', y= 'Energy(J)', data=df_llama7b_measurement)
plt.xlabel('Device',size=20,weight = 'bold')
plt.ylabel('Energy (Jules)',size=20,weight = 'bold')
ax = plt.gca()
for spine in ax.spines.values():
    spine.set_linewidth(2)  # You can adjust the linewidth as needed
ax.tick_params(axis='both', which='major', labelsize=18, width=2)  # You can adjust the size and width as needed
ax.tick_params(axis='both', which='minor', labelsize=16, width=1.5)  # For minor ticks if needed

# Uncomment to set y-axis limit if required
# plt.ylim(800,)

# Show plot
plt.show()
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# Plot KDE for both datasets with different alpha values
sns.kdeplot(df_llama7b_l4['Energy(J)'],color='tab:blue', label='L4')
sns.kdeplot(df_llama7b_a100['Energy(J)'], color='chocolate', label='A100')

# Add labels and title
plt.xlabel('Energy(J)',size=20,weight = 'bold')
plt.ylabel('Probability Density',size=20,weight = 'bold')

ax = plt.gca()
for spine in ax.spines.values():
    spine.set_linewidth(2)  # You can adjust the linewidth as needed
ax.tick_params(axis='both', which='major', labelsize=18, width=2)  # You can adjust the size and width as needed
ax.tick_params(axis='both', which='minor', labelsize=16, width=1.5)  # For minor ticks if needed

# Uncomment to set y-axis limit if required
# plt.ylim(800,)



plt.grid()
plt.legend( prop={'size': 14, 'weight': 'bold'})  # Increase size and make bold
# Show plot
plt.show()


In [ ]:
# Plot CDF for each dataset with distinct colors, line styles, and markers
sns.ecdfplot(df_llama7b_l4['Energy(J)'], label='L4', color='blue', linestyle='-',  linewidth=3)
sns.ecdfplot(df_llama7b_a100['Energy(J)'], label='A100', color='green', linestyle='--',  linewidth=3)

# Add labels and title
plt.xlabel('Energy (J)', fontsize=18,weight = 'bold')
plt.ylabel('CDF', fontsize=18,weight = 'bold')
ax = plt.gca()
for spine in ax.spines.values():
    spine.set_linewidth(2)  # You can adjust the linewidth as needed
ax.tick_params(axis='both', which='major', labelsize=18, width=2)  # You can adjust the size and width as needed
ax.tick_params(axis='both', which='minor', labelsize=16, width=2)  # For minor ticks if needed
plt.grid()
plt.legend( prop={'size': 14, 'weight': 'bold'})  # Increase size and make bold

# Show plot
plt.show()


In [ ]:
# Plot CDF for each dataset with distinct colors, line styles, and markers
sns.ecdfplot(df_llama7b_l4['duration'], label='L4', color='blue', linestyle='-',  linewidth=3)
sns.ecdfplot(df_llama7b_a100['duration'], label='A100', color='green', linestyle='--',  linewidth=3)

# Add labels and title
plt.xlabel('Duration (s)', fontsize=18,weight = 'bold')
plt.ylabel('CDF', fontsize=18,weight = 'bold')
ax = plt.gca()
for spine in ax.spines.values():
    spine.set_linewidth(2)  # You can adjust the linewidth as needed
ax.tick_params(axis='both', which='major', labelsize=18, width=2)  # You can adjust the size and width as needed
ax.tick_params(axis='both', which='minor', labelsize=16, width=2)  # For minor ticks if needed
plt.grid()
plt.legend( prop={'size': 14, 'weight': 'bold'})  # Increase size and make bold

# Show plot
plt.show()

In [60]:
df_llama7b_colab = pd.concat([df_llama7b_l4,df_llama7b_a100])

In [51]:
df_llama7b_l4['duration'].mean(),df_llama7b_l4['energy_consumed'].mean()*3.6e+6*1.2,df_llama7b_l4['gpu_power'].mean(),

(19.528528656959534, 3142.859778935061, 71.42219626314719)

In [13]:
df_llama7b_l4['gpu_power'].mean(),df_llama7b_l4['cpu_power'].mean(),df_llama7b_l4['ram_power'].mean()

(71.42219626314719, 42.5, 19.86054039001465)

In [14]:
df_llama7b_l4['gpu_energy'].mean()*3.6e+6,df_llama7b_l4['cpu_energy'].mean()*3.6e+6,df_llama7b_l4['ram_energy'].mean()*3.6e+6

(1401.3474410768827, 829.9602946399457, 387.7420800621671)

In [52]:
df_llama7b_a100['duration'].mean(),df_llama7b_a100['energy_consumed'].mean()*3.6e+6*1.2,df_llama7b_a100['gpu_power'].mean(),

(12.803307657241822, 2874.46402605062, 112.50856846942959)

In [41]:
df_llama7b_a100['gpu_energy'].mean()*3.6e+6,df_llama7b_a100['cpu_energy'].mean()*3.6e+6,df_llama7b_a100['ram_energy'].mean()*3.6e+6

(1450.5954404752883, 544.1387470721165, 400.6525008279565)

In [47]:
df_llama7b_a100['gpu_power'].mean(),df_llama7b_a100['cpu_power'].mean(),df_llama7b_a100['ram_power'].mean()

(112.50856846942959, 42.5, 31.30389261245728)

In [ ]:
# sns.barplot(data=df_llama7b_measurement, x='Source', y='Energy(J)', hue='GPU')
# plt.legend(ncol=(2,2), loc=(0.1, 1.05), prop={'size': 20, 'weight': 'bold'})  # Increase size and make bold
plt.bar(['L4','A100','S23','S24','X Elite'],
        [df_llama7b_l4['Energy(J)'].mean(),df_llama7b_a100['Energy(J)'].mean()],
        color=['tab:blue','tab:orange','tab:red','tab:green'])
plt.xlabel('Device',size=20,weight = 'bold')
plt.ylabel('Energy (Jules)',size=20,weight = 'bold')
ax = plt.gca()
for spine in ax.spines.values():
    spine.set_linewidth(2)  # You can adjust the linewidth as needed
ax.tick_params(axis='both', which='major', labelsize=18, width=2)  # You can adjust the size and width as needed
ax.tick_params(axis='both', which='minor', labelsize=16, width=1.5)  # For minor ticks if needed

# Uncomment to set y-axis limit if required
# plt.ylim(800,)

# Show plot
plt.show()
plt.show()

In [ ]:
# sns.barplot(data=combined_df, x='GPU', y='duration', hue='Source')
plt.bar(['L4','A100','S23','S24','X Elite'],
        [df_llama7b_l4['duration'].mean(),df_llama7b_a100['duration'].mean()],
        color=['tab:blue','tab:orange','tab:red','tab:green'])
# plt.legend(ncol=3, loc=(0.05, 1.05), prop={'size': 20, 'weight': 'bold'})  # Increase size and make bold
plt.xlabel('Device',size=20,weight = 'bold')
plt.ylabel('Duration (Sec)',size=20,weight = 'bold')
# plt.ylim(800,)
ax = plt.gca()
for spine in ax.spines.values():
    spine.set_linewidth(2)  # You can adjust the linewidth as needed
ax.tick_params(axis='both', which='major', labelsize=18, width=2)  # You can adjust the size and width as needed
ax.tick_params(axis='both', which='minor', labelsize=16, width=1.5)  # For minor ticks if needed

# Uncomment to set y-axis limit if required
# plt.ylim(800,)

# Show plot
plt.show()
plt.show()

In [ ]:
# sns.barplot(data=combined_df, x='GPU', y='Throughput (Token/s)', hue='Source')
plt.bar(['L4','A100','S23','S24','X Elite'],
        [df_llama7b_l4['Throughput (Token/s)'].mean(),
         df_llama7b_a100['Throughput (Token/s)'].mean()],
        color=['tab:blue','chocolate','firebrick','darkgreen','darkmagenta'])
plt.xlabel('Device',size=20,weight = 'bold')
plt.ylabel('Throughput (Token/s)',size=20,weight = 'bold')
# plt.ylim(800,)
ax = plt.gca()
for spine in ax.spines.values():
    spine.set_linewidth(2)  # You can adjust the linewidth as needed
ax.tick_params(axis='both', which='major', labelsize=18, width=2)  # You can adjust the size and width as needed
ax.tick_params(axis='both', which='minor', labelsize=16, width=1.5)  # For minor ticks if needed

# Uncomment to set y-axis limit if required
# plt.ylim(800,)

# Show plot
plt.show()
plt.show()

In [ ]:
# sns.barplot(data=combined_df, x='GPU', y='Energy/Token (J/Token)', hue='Source')
# plt.legend(ncol=3, loc=(0.05, 1.05), prop={'size': 20, 'weight': 'bold'})  # Increase size and make bold
plt.bar(['L4','A100','S23','S24','X Elite'],
        [df_llama7b_l4['Energy/Token (J/Token)'].mean(),
         df_llama7b_a100['Energy/Token (J/Token)'].mean()],
       color=['tab:blue','chocolate','firebrick','darkgreen','darkmagenta'])
plt.xlabel('Device',size=20,weight = 'bold')
plt.ylabel('Energy/Token (J/Token)',size=20,weight = 'bold')
# plt.ylim(800,)
ax = plt.gca()
for spine in ax.spines.values():
    spine.set_linewidth(2)  # You can adjust the linewidth as needed
ax.tick_params(axis='both', which='major', labelsize=18, width=2)  # You can adjust the size and width as needed
ax.tick_params(axis='both', which='minor', labelsize=16, width=1.5)  # For minor ticks if needed

# Uncomment to set y-axis limit if required
# plt.ylim(800,)

# Show plot
plt.show()
plt.show()